In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mnist_root = '../data/'
# Training dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=mnist_root, train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ])), batch_size=64, shuffle=True, num_workers=8)
# Test dataset
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=mnist_root, train=False, transform=transforms.Compose([
        transforms.ToTensor(),
    ])), batch_size=64, shuffle=True, num_workers=4)

fgsm_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=mnist_root, train=False, transform=transforms.Compose([
        transforms.ToTensor(),
    ])), batch_size=1, shuffle=True, num_workers=4)

print(len(train_loader), len(test_loader))

from utils import train, test, train_pgd, train_pgd_plus
# from utils_fix import train, test, train_pgd
from functools import partial
train_mnist_adv = partial(train_pgd, train_loader=train_loader, eps=0.1, niter=10, alpha=0.004)
test_mnist = partial(test, data_loader=test_loader)

938 157


In [3]:
# lr decay every 15 epoch
def Pipeline(sdir, model, optimizer, name='', nepoch=50, bep=1, patience=15, 
             tloss=0.1, tacc=99, crit=nn.CrossEntropyLoss()):
    if not os.path.isdir(sdir):
        os.makedirs(sdir)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=patience)
    for epoch in range(bep, nepoch+bep):
        train_mnist_adv(model, epoch=epoch, optimizer=optimizer, crit=crit)
        vloss,vacc = test_mnist(model,test_crit=crit)
        scheduler.step()
        if tloss>vloss or vacc>tacc:    # save best model
            torch.save(model.state_dict(), '%s/%s_ep-%02d_vloss-%.4f_vacc-%.2f.pt' % (sdir, name, epoch, vloss, vacc))
            tloss = vloss; tacc = vacc
    return tloss, tacc

In [4]:
from torch._jit_internal import weak_module, weak_script_method, List

@weak_module
class condNorm(nn.Module):
    def __init__(self, planes=64, std=1.0, minl=1.0, eps=1e-5):
        super().__init__()
        self.planes = planes
        self.std = std
        self.minl = minl
        self.eps = eps
    
    def forward(self, x):
        # x: N x C x H x W,  4D Tensor
        x = x - x.mean([2,3], keepdim=True)
        norm = torch.sqrt(torch.mean(x**2, dim=[2,3], keepdim=True) + self.eps)
        x = self.std * x / torch.max(norm, self.minl*torch.ones_like(norm))
        return x
    
from torch.autograd import Variable
import torch.optim as optim
%matplotlib inline

def convert_np(inp, group=8):
    inp = inp.detach().cpu().squeeze()
    n, h, w = inp.shape
    t = n // group
    g = group
    out = [inp[i:i+g].view(h*g,w) for i in range(0, n, g)]
    out = torch.cat(out, -1)
    out = np.clip(out.detach().cpu().numpy(), 0, 1)
    return out


def show_batch(inp1, inp2, dpi=130):
    out1 = convert_np(inp1)
    out2 = convert_np(inp2)
    fig, ax = plt.subplots(1,2, dpi=dpi)
    ax[0].imshow(out1, cmap='gray')
    ax[1].imshow(out2, cmap='gray')
    plt.show()
    

def advs_pgd(model, X, y, epsilon, niters=100, alpha=0.01): 
    out = model(X)
    ce = nn.CrossEntropyLoss()(out, y)
    err = (out.data.max(1)[1] != y.data).float().sum()  / X.size(0)

    X_pgd = Variable(X.data, requires_grad=True)
    for i in range(niters): 
        opt = optim.Adam([X_pgd], lr=1e-3)
        opt.zero_grad()
        loss = nn.CrossEntropyLoss()(model(X_pgd), y)
        loss.backward()
        eta = alpha*X_pgd.grad.data.sign()
        X_pgd = Variable(X_pgd.data + eta, requires_grad=True)
        
        # adjust to be within [-epsilon, epsilon]
        eta = torch.clamp(X_pgd.data - X.data, -epsilon, epsilon)
        X_pgd = Variable(X.data + eta, requires_grad=True)
        
    err_pgd = (model(X_pgd).data.max(1)[1] != y.data).float().sum() / X.size(0)
    print("err: %.2f%% %.2f%%" % (err*100, err_pgd*100))
    return X_pgd

# Test resnet

## pure conv

In [5]:
from sim_model.res_free import ResNet18

model = ResNet18().to(device)
print('Number of Parameters: ', sum([p.numel() for p in model.parameters()]))
print('Training Parameters: ', sum([p.numel() for p in model.parameters() if p.requires_grad]))
model

Number of Parameters:  11163210
Training Parameters:  11163210


In [ ]:
sdir = './runs/tmp_time_test'
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
tloss, tacc = Pipeline(sdir, model, optimizer, 'res18_free', 45)

Train Epoch: 1 [29952/60000 (50%)]	Avgloss: 0.578816	Time: 102.7
Train Epoch: 1 [59904/60000 (100%)]	Avgloss: 0.104882	Time: 206.6
                                                   
Test set: Avg loss: 0.0528 |  Accuracy: 98.46 | 

Train Epoch: 2 [29952/60000 (50%)]	Avgloss: 0.079633	Time: 103.0
Train Epoch: 2 [59904/60000 (100%)]	Avgloss: 0.076035	Time: 205.5
                                                   
Test set: Avg loss: 0.0503 |  Accuracy: 98.75 | 



In [ ]:
sdir = './runs/tmp_time_test'
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
tloss, tacc = Pipeline(sdir, model, optimizer, 'res18_free', 45)

Train Epoch: 1 [29952/60000 (50%)]	Avgloss: 0.386317	Time: 105.4
Train Epoch: 1 [59904/60000 (100%)]	Avgloss: 0.101694	Time: 210.2
                                                   
Test set: Avg loss: 0.0449 |  Accuracy: 98.58 | 

Train Epoch: 2 [29952/60000 (50%)]	Avgloss: 0.078865	Time: 105.2
Train Epoch: 2 [59904/60000 (100%)]	Avgloss: 0.069506	Time: 210.3
                                                   
Test set: Avg loss: 0.0311 |  Accuracy: 99.06 | 



## +bn 

In [5]:
from sim_model.res_new import ResNet18

model = ResNet18().to(device)  # BN default
print('Number of Parameters: ', sum([p.numel() for p in model.parameters()]))
print('Training Parameters: ', sum([p.numel() for p in model.parameters() if p.requires_grad]))
model

Number of Parameters:  11172682
Training Parameters:  11172682


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
  )
  (layer2): Sequ

In [ ]:
sdir = './runs/tmp_time_test'
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
tloss, tacc = Pipeline(sdir, model, optimizer, 'res18_bn', 45)

Train Epoch: 1 [29952/60000 (50%)]	Avgloss: 0.188499	Time: 121.2
Train Epoch: 1 [59904/60000 (100%)]	Avgloss: 0.080126	Time: 241.3
                                                   
Test set: Avg loss: 0.0525 |  Accuracy: 98.32 | 

Train Epoch: 2 [29952/60000 (50%)]	Avgloss: 0.065976	Time: 120.6
Train Epoch: 2 [59904/60000 (100%)]	Avgloss: 0.062390	Time: 240.7
                                                   
Test set: Avg loss: 0.0370 |  Accuracy: 98.86 | 

Train Epoch: 3 [29952/60000 (50%)]	Avgloss: 0.053412	Time: 120.4
Train Epoch: 3 [59904/60000 (100%)]	Avgloss: 0.058397	Time: 240.4
                                                   
Test set: Avg loss: 0.0430 |  Accuracy: 98.77 | 

Train Epoch: 4 [29952/60000 (50%)]	Avgloss: 0.049310	Time: 120.2
Train Epoch: 4 [59904/60000 (100%)]	Avgloss: 0.048004	Time: 240.0
                                                   
Test set: Avg loss: 0.0480 |  Accuracy: 98.56 | 

Train Epoch: 5 [29952/60000 (50%)]	Avgloss: 0.042410	Time: 120.0
Tra

In [ ]:
sdir = './runs/tmp_time_test'
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
tloss, tacc = Pipeline(sdir, model, optimizer, 'res18_bn', 45)

Train Epoch: 1 [29952/60000 (50%)]	Avgloss: 0.188499	Time: 121.2
Train Epoch: 1 [59904/60000 (100%)]	Avgloss: 0.080126	Time: 241.3
                                                   
Test set: Avg loss: 0.0525 |  Accuracy: 98.32 | 

Train Epoch: 2 [29952/60000 (50%)]	Avgloss: 0.065976	Time: 120.6
Train Epoch: 2 [59904/60000 (100%)]	Avgloss: 0.062390	Time: 240.7
                                                   
Test set: Avg loss: 0.0370 |  Accuracy: 98.86 | 

Train Epoch: 3 [29952/60000 (50%)]	Avgloss: 0.053412	Time: 120.4
Train Epoch: 3 [59904/60000 (100%)]	Avgloss: 0.058397	Time: 240.4
                                                   
Test set: Avg loss: 0.0430 |  Accuracy: 98.77 | 

Train Epoch: 4 [29952/60000 (50%)]	Avgloss: 0.049310	Time: 120.2
Train Epoch: 4 [59904/60000 (100%)]	Avgloss: 0.048004	Time: 240.0
                                                   
Test set: Avg loss: 0.0480 |  Accuracy: 98.56 | 

Train Epoch: 5 [29952/60000 (50%)]	Avgloss: 0.042410	Time: 120.0
Tra

## +cn 

In [5]:
from sim_model.res_new import ResNet18

# @weak_module
class condNorm(nn.Module):
    def __init__(self, planes=64, std=1.0, minl=1.0, eps=1e-5):
        super().__init__()
        self.planes = planes
        self.std = std
        self.minl = minl
        self.eps = eps
    
    def forward(self, x):
        # x: N x C x H x W,  4D Tensor
        x = x - x.mean([0,2,3], keepdim=True)
        norm = torch.sqrt(torch.mean(x**2, dim=[0,2,3], keepdim=True) + self.eps)
        x = self.std * x / torch.max(norm, self.minl*torch.ones_like(norm))
        return x
    
model = ResNet18(normlayer=condNorm).to(device)  # BN default
print('Number of Parameters: ', sum([p.numel() for p in model.parameters()]))
print('Training Parameters: ', sum([p.numel() for p in model.parameters() if p.requires_grad]))
model

Number of Parameters:  11165002
Training Parameters:  11165002


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): condNorm()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): condNorm()
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): condNorm()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): condNorm()
      (shortcut): Sequential()
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): condNorm()
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): condNorm()
      (sho

In [ ]:
sdir = './runs/tmp_time_test'
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
tloss, tacc = Pipeline(sdir, model, optimizer, 'res18_bcn', 45)

Train Epoch: 1 [29952/60000 (50%)]	Avgloss: 0.191948	Time: 176.8
Train Epoch: 1 [59904/60000 (100%)]	Avgloss: 0.086995	Time: 353.0
                                                   
Test set: Avg loss: 0.0607 |  Accuracy: 98.15 | 

Train Epoch: 2 [29952/60000 (50%)]	Avgloss: 0.073682	Time: 178.1
Train Epoch: 2 [59904/60000 (100%)]	Avgloss: 0.077037	Time: 354.6
                                                   
Test set: Avg loss: 0.0364 |  Accuracy: 98.99 | 

Train Epoch: 3 [29952/60000 (50%)]	Avgloss: 0.073153	Time: 178.9
Train Epoch: 3 [59904/60000 (100%)]	Avgloss: 0.067797	Time: 357.1
                                                   
Test set: Avg loss: 0.0377 |  Accuracy: 98.73 | 

Train Epoch: 4 [29952/60000 (50%)]	Avgloss: 0.066590	Time: 179.5
Train Epoch: 4 [59904/60000 (100%)]	Avgloss: 0.066165	Time: 358.4
                                                   
Test set: Avg loss: 0.0402 |  Accuracy: 98.73 | 



# fix inplace +bcn(cuda update) 

In [5]:
from sim_model.res_new_fix_inplace import ResNet18
from cond_norm import batchCondNorm

model = ResNet18(normlayer=batchCondNorm).to(device)  
print('Number of Parameters: ', sum([p.numel() for p in model.parameters()]))
# print('Training Parameters: ', sum([p.numel() for p in model.parameters() if p.requires_grad]))
model

Number of Parameters:  11172682
Training Parameters:  11172682


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): TCondNorm(planes=64, minl=1.0, epsilon=1e-05)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): TCondNorm(planes=64, minl=1.0, epsilon=1e-05)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): TCondNorm(planes=64, minl=1.0, epsilon=1e-05)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): TCondNorm(planes=64, minl=1.0, epsilon=1e-05)
      (shortcut): Sequential()
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): 

In [6]:
sdir = './runs/tmp_time_test'
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
tloss, tacc = Pipeline(sdir, model, optimizer, 'res18_bcn_ext1', 4)

Train Epoch: 1 [29952/60000 (50%)]	Avgloss: 0.186916	Time: 124.9
Train Epoch: 1 [59904/60000 (100%)]	Avgloss: 0.082959	Time: 250.0
                                                   
Test set: Avg loss: 0.0492 |  Accuracy: 98.47 | 

Train Epoch: 2 [29952/60000 (50%)]	Avgloss: 0.066446	Time: 125.2
Train Epoch: 2 [59904/60000 (100%)]	Avgloss: 0.073373	Time: 249.4
                                                   
Test set: Avg loss: 0.0415 |  Accuracy: 98.66 | 

Train Epoch: 3 [29952/60000 (50%)]	Avgloss: 0.058973	Time: 124.6
Train Epoch: 3 [59904/60000 (100%)]	Avgloss: 0.060851	Time: 248.5
                                                   
Test set: Avg loss: 0.0377 |  Accuracy: 98.91 | 

Train Epoch: 4 [29952/60000 (50%)]	Avgloss: 0.053337	Time: 124.3
Train Epoch: 4 [59904/60000 (100%)]	Avgloss: 0.050309	Time: 247.8
                                                   
Test set: Avg loss: 0.0312 |  Accuracy: 99.13 | 



In [6]:
# before: 253-256
sdir = './runs/tmp_time_test'
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
tloss, tacc = Pipeline(sdir, model, optimizer, 'res18_bcn_ext1', 20)

Train Epoch: 1 [29952/60000 (50%)]	Avgloss: 0.190281	Time: 124.6
Train Epoch: 1 [59904/60000 (100%)]	Avgloss: 0.085715	Time: 248.6
                                                   
Test set: Avg loss: 0.0586 |  Accuracy: 98.23 | 

Train Epoch: 2 [29952/60000 (50%)]	Avgloss: 0.077424	Time: 125.7
Train Epoch: 2 [59904/60000 (100%)]	Avgloss: 0.065668	Time: 249.8
                                                   
Test set: Avg loss: 0.0464 |  Accuracy: 98.59 | 

Train Epoch: 3 [29952/60000 (50%)]	Avgloss: 0.062373	Time: 124.5
Train Epoch: 3 [59904/60000 (100%)]	Avgloss: 0.056342	Time: 248.5
                                                   
Test set: Avg loss: 0.0622 |  Accuracy: 98.32 | 

Train Epoch: 4 [29952/60000 (50%)]	Avgloss: 0.052414	Time: 124.3
Train Epoch: 4 [59904/60000 (100%)]	Avgloss: 0.055329	Time: 248.0
                                                   
Test set: Avg loss: 0.0327 |  Accuracy: 99.00 | 

Train Epoch: 5 [29952/60000 (50%)]	Avgloss: 0.046769	Time: 123.7
Tra

# more layer

In [5]:
# resnet34
from sim_model.res_free import ResNet34

model = ResNet34().to(device)
print('Number of Parameters: ', sum([p.numel() for p in model.parameters()]))
print('Training Parameters: ', sum([p.numel() for p in model.parameters() if p.requires_grad]))
model

Number of Parameters:  21263946
Training Parameters:  21263946


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (shortcut): Sequential()
    )
    (2): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (shortcut): Sequential()
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
     

In [ ]:
sdir = './runs/tmp_time_test'
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
tloss, tacc = Pipeline(sdir, model, optimizer, 'res34_pure', 45)

Train Epoch: 1 [29952/60000 (50%)]	Avgloss: 0.825230	Time: 172.4
Train Epoch: 1 [59904/60000 (100%)]	Avgloss: 0.130399	Time: 344.3
                                                   
Test set: Avg loss: 0.0564 |  Accuracy: 98.56 | 

Train Epoch: 2 [29952/60000 (50%)]	Avgloss: 0.095351	Time: 172.9
Train Epoch: 2 [59904/60000 (100%)]	Avgloss: 0.086182	Time: 345.0
                                                   
Test set: Avg loss: 0.0514 |  Accuracy: 98.72 | 



In [ ]:
# bn
from sim_model.res_new_fix_inplace import ResNet34
from cond_norm import batchCondNorm

model = ResNet34().to(device)
print('Number of Parameters: ', sum([p.numel() for p in model.parameters()]))
print(model)
sdir = './runs/tmp_time_test'
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
tloss, tacc = Pipeline(sdir, model, optimizer, 'res34_bn', 4)

Number of Parameters:  21280842
ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequent

                                                   
Test set: Avg loss: 0.0577 |  Accuracy: 98.25 | 



In [5]:
# bcn
from sim_model.res_new_fix_inplace import ResNet34
from cond_norm import batchCondNorm

model = ResNet34(normlayer=batchCondNorm).to(device)
print('Number of Parameters: ', sum([p.numel() for p in model.parameters()]))
print(model)
sdir = './runs/tmp_time_test'
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
tloss, tacc = Pipeline(sdir, model, optimizer, 'res34_bcn_ext1', 20)

Number of Parameters:  21280842
ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): TCondNorm(planes=64, minl=1.0, epsilon=1e-05)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): TCondNorm(planes=64, minl=1.0, epsilon=1e-05)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): TCondNorm(planes=64, minl=1.0, epsilon=1e-05)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): TCondNorm(planes=64, minl=1.0, epsilon=1e-05)
      (shortcut): Sequential()
    )
    (2): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1

Train Epoch: 7 [59904/60000 (100%)]	Avgloss: 0.037854	Time: 416.5
                                                   
Test set: Avg loss: 0.0336 |  Accuracy: 98.96 | 

Train Epoch: 8 [29952/60000 (50%)]	Avgloss: 0.031720	Time: 207.7
Train Epoch: 8 [59904/60000 (100%)]	Avgloss: 0.032709	Time: 414.3
                                                   
Test set: Avg loss: 0.0267 |  Accuracy: 99.15 | 

Train Epoch: 9 [29952/60000 (50%)]	Avgloss: 0.027264	Time: 208.0
Train Epoch: 9 [59904/60000 (100%)]	Avgloss: 0.032461	Time: 415.5
                                                   
Test set: Avg loss: 0.0227 |  Accuracy: 99.25 | 

Train Epoch: 10 [29952/60000 (50%)]	Avgloss: 0.026832	Time: 208.2
Train Epoch: 10 [59904/60000 (100%)]	Avgloss: 0.028370	Time: 415.8
                                                   
Test set: Avg loss: 0.0208 |  Accuracy: 99.40 | 

Train Epoch: 11 [29952/60000 (50%)]	Avgloss: 0.022024	Time: 212.7
Train Epoch: 11 [59904/60000 (100%)]	Avgloss: 0.027637	Time: 422.

In [5]:
# bcn torch
from sim_model.res_new_fix_inplace import ResNet34

# @weak_module
class condNorm(nn.Module):
    def __init__(self, planes=64, minl=1.0, eps=1e-5):
        super().__init__()
        self.planes = planes
        self.minl = minl
        self.eps = eps
    
    def forward(self, x):
        # x: N x C x H x W,  4D Tensor
        x = x - x.mean([0,2,3], keepdim=True)
        norm = torch.sqrt(torch.mean(x**2, dim=[0,2,3], keepdim=True) + self.eps)
        x = x / torch.max(norm, self.minl*torch.ones_like(norm))
        return x
    
model = ResNet34(normlayer=condNorm).to(device)
print('Number of Parameters: ', sum([p.numel() for p in model.parameters()]))
print(model)
sdir = './runs/tmp_time_test'
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
tloss, tacc = Pipeline(sdir, model, optimizer, 'res34_bcn_th', 4)

Number of Parameters:  21265738
ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): condNorm()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): condNorm()
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): condNorm()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): condNorm()
      (shortcut): Sequential()
    )
    (2): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): condNorm()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): condNorm()
      (sh